In [1]:
from sklearn.cluster import MiniBatchKMeans
import datasets
import utils
import tester
import models
import numpy as np
from scipy import spatial

In [43]:
def calculateSSD(model, X):
    cluster_sum = {i:0 for i in range(model.n_clusters)}
    for i, x in enumerate(X):
        label = model.labels_[i]
        cluster_sum[label] += utils.distance(x, model.cluster_centers_[label])**2

    return sum(cluster_sum.values())

def createKmeans(data, n_clusters):
    model_list = [MiniBatchKMeans(n_clusters=n_clusters, init_size=n_clusters*3) for _ in data.cd]
    for y in data.cd:
        model_list[y].fit(data.cd[y])
        
    ssd = sum([calculateSSD(model_list[y], data.cd[y]) for y in data.cd])
    return model_list, ssd

In [44]:
data = datasets.credit_cards(normalized=True)

x = [createKmeans(data, 1000) for _ in range(10)]

model_list = min(x, key=lambda k : k[1])[0]

In [4]:
X = []
Y = []
for y in sorted(data.cd):
    X.extend(model_list[y].cluster_centers_)
    Y.extend([y] * len(model_list[y].cluster_centers_))

custom_dataset = datasets.Dataset(np.array(X), np.array(Y), normalized=True)

In [35]:
model_tester = tester.Tester(custom_dataset, models.KNN(13))
model_tester.test(transform=utils.LDA())

In [36]:
statistics = model_tester.statistics()
statistics.print_all()

Confusion Matrix:


,0,1
0,13972.0,6028.0
1,8137.0,11863.0


Test Accuracy:


Mean,Median,Min,Max,STD,Specificity,Sensibility
0.645875,0.64625,0.5825,0.7125,0.022966,0.63196,0.663071
